In [ ]:
pip install -r requirements.txt

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [5]:
base = pd.read_csv('petr4_treinamento.csv') # Importando a base de dados
base = base.dropna() # Removendo valores nulos

In [6]:
base_treinamento = base.iloc[:, 1:7].values # Pegando apenas a coluna de abertura

In [7]:
normalizador = MinMaxScaler(feature_range=(0,1)) # Normalizando os dados
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [ ]:
normalizador_previsao = MinMaxScaler(feature_range=(0,1)) # Normalizando os dados
normalizador_previsao.fit_transform(base_treinamento[:,0:1])

In [10]:
X = [] # Criando a lista de previsores
y = [] # Criando a lista de valores reais
for i in range(90, 1242): # Pegando os 90 primeiros valores para prever o 91
    X.append(base_treinamento_normalizada[i-90:i, 0:6]) # Pegando os 90 valores anteriores
    y.append(base_treinamento_normalizada[i, 0]) # Pegando o valor atual

X, y = np.array(X), np.array(y) # Transformando em array

In [ ]:
regressor = Sequential() #iniciar a rede neural

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (X.shape[1], 6))) #camada de entrada com 100 neuronios e 90 camadas de entrada (X.shape[1] = 90)
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50, return_sequences = True)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50, return_sequences = True)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(Dense(units = 1, activation = 'linear')) #camada de saida com 1 neuronio

regressor.summary() #resumo da rede neural

In [12]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mean_absolute_error']) #compilando a rede neural

In [13]:
es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=True) #parar o treinamento quando a rede neural parar de aprender
rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1) #diminuir a taxa de aprendizado quando a rede neural parar de aprender
mcp = ModelCheckpoint(filepath='pesos.keras', monitor='loss', save_best_only=True, verbose=1) #salvar os melhores pesos


In [ ]:
regressor.fit(X, y, epochs = 100, batch_size = 32, callbacks=[es, rlr, mcp]) #treinando a rede neural 

In [24]:
base_teste = pd.read_csv('petr4_teste.csv') #importando a base de dados de teste
y_teste = base_teste.iloc[:, 1:2].values #pegando a coluna de abertura
frames = [base, base_teste] #juntando as duas bases de dados
base_completa = pd.concat(frames) #juntando as duas bases de dados

In [25]:
base_completa = base_completa.drop('Date', axis = 1) #removendo a coluna de data

In [26]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values #pegando os 90 valores anteriores ao primeiro valor da base de teste

In [27]:
entradas = normalizador.transform(entradas) #normalizando os valores

In [ ]:
X_teste = [] #criando a lista de previsores
for i in range(90, 112): #pegando os 90 primeiros valores para prever o 91
    X_teste.append(entradas[i-90:i, 0:6]) #pegando os 90 valores anteriores
X_teste = np.array(X_teste) #transformando em array

In [ ]:
previsoes = regressor.predict(X_teste) #fazendo as previsoes

In [37]:
previsoes = normalizador_previsao.inverse_transform(previsoes) #desnormalizando os valores

In [ ]:
previsoes

In [ ]:
y_teste

In [ ]:
previsoes.mean()

In [ ]:
y_teste.mean()

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_teste, previsoes)